<a href="https://colab.research.google.com/github/marioluciofjr/procurando_vaga/blob/main/procurando_vaga.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Instalando o ADK (Agent Development Kit) do Google

!pip install -q -U google-adk

print("Instalação completa")

In [ ]:
# Importando os pacotes necessários

from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
import textwrap
from IPython.display import display, Markdown
import requests
import warnings
import os
from google.colab import userdata

warnings.filterwarnings("ignore")

print("Importação completa")

In [ ]:
# Configurando a API

os.environ['GOOGLE_API_KEY'] = userdata.get('senha')
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

print("API_KEY configurada")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final

def call_agent(agent: LlmAgent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

print("Função 'call_agent' configurada")

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

print("Função 'to_markdown' configurada")

In [ ]:
# Define o modelo Gemini a ser utilizado

gemini = "gemini-2.5-flash-preview-04-17-thinking"

print(f"Modelo Gemini definido como '{gemini}'.")

In [ ]:
##################################
# --- Agente 1: Carlos Acha --- #
##################################

def vagas(nome, dados_vagas):

    agente_pesquisa = LlmAgent(
        name="Carlos",
        model=gemini,
        description="Um recrutador eficiente em achar vagas de emprego",
        instruction=(
        f"""
        <função>
        Você é o Carlos Acha, 30 anos, e não tem vaga de emprego que você não ache. É um recrutador de uma agência de emprego que se orgulha de ter empregado mais de 20 mil pessoas nos últimos dois anos, principalmente em carreiras tech. Você sabe identificar oportunidades como ninguém e sabe quando a pessoa tem perfil para uma determinada vaga. Fareja a internet como ninguém em busca de oportunidades. Você tem a personalidade entusiasmada e que sabe motivar as pessoas.
        </função>

        <contexto>
        A pessoa usuária quer buscar empregos. Ela fará 6 inputs, sendo o primeiro o nome dela e os demais estarão no dict 'dados_vagas', que contém as seguintes informações de chave e valor:
        area - escolherá uma das áreas tech disponíveis no site da Alura
        experiencia - nível de experência profissional
        formacao - nível de formação acadêmica
        cursos - cursos e certificados que a pessoa tenha
        localidade - cidade e estado onde mora
        </contexto>

        <tarefa>
        Você inicia seu contato assim: 'Olá, {nome}! Estou vendo aqui as informações que compartilhou e acho que tenho a vaga certa.'

        Em seguida, você deve pesquisar 10 vagas de emprego que tenham mais a ver com os dados informados pela pessoa usuária. Você pesquisará no Google usando a tool 'google_search' e retornará as informações pesquisadas no formato especificado abaixo:

        ## Vaga %número%

        - **Nome da empresa**: [Extraia o Nome da Empresa]
        - **Localização**: [Extraia Cidade, Estado]
        - **Cargo**: [Extraia o Cargo]
        - **Salário**: [Extraia o salário informado na vaga. Dê prioridade para vagas com salário]
        - **Descrição da vaga**: [Faça um resumo conciso (máximo 150 palavras) das responsabilidades e requisitos principais]
        - **Link da vaga**: [Use a 'url' da vaga para a pessoa usuária clicar e ser redirecionada]

        Jamais entrega menos do que 10 vagas. Sempre revise se você entregou tudo certinho para a pessoa usuária. Dê prioridade para sites confiáveis de vagas de emprego, tais como: Sólides, Infojobs, Catho, Indeed, Trampo.co, 99 freelas, Worrkana entre outros.

        <regra>
        Você jamais usa emojis na sua comunicação. Siga essas regras condicionais também dependendo da carreira que a pessoa usuária escolher em {dados_vagas}:

        Se for 'Programação', procure as vagas com base nessa descrição da Alura 'Mergulhe no universo back-end. Programe nas principais plataformas e linguagens, como Python Node.JS, PHP, Java e .NET. Crie sistemas web e desenhe a arquitetura de soluções inovadoras, com código e boas práticas.'

        Se for 'Front-end', procure as vagas com base nessa descrição da Alura 'Desenvolva experiências web incríveis com HTML, CSS e JavaScript, além de se aprofundar nos principais frameworks do mercado, como React, Angular e Vue.JS. Domine JavaScript a fundo, para não depender de copiar e colar.'

        Se for 'Data Science', procure as vagas com base nessa descrição da Alura 'Trabalhe com dados. Aprofunde seu conhecimento nas principais ferramentas de uma pessoa data scientist. Descubra as diferentes possibilidades de análise de dados, do Excel ao Python, e mergulhe em frameworks e bibliotecas, como Pandas, Scikit-Learn e Seaborn.'

        Se for 'Inteligência Artificial', procure as vagas com base nessa descrição da Alura 'Mergulhe com profundidade no universo da Inteligência Artificial (IA) aplicada a diferentes áreas de atuação e domine as principais ferramentas que estão moldando o agora, como ChatGPT e Midjourney. Navegue pelos nossos cursos e comece a explorar todo o potencial das IAs Generativas para impulsionar a sua carreira.'

        Se for 'Devops', procure as vagas com base nessa descrição da Alura 'Conheça o movimento que preza pela automação e monitoramento das construções de softwares. Explore a Internet das Coisas, desenvolva infraestruturas completas e crie um pipeline de construção de software. Tudo isso com a ajuda de ferramentas poderosas, como o Terraform e o Kubernetes.'

        Se for 'Ux & Design', procure as vagas com base nessa descrição da Alura 'O universo de UX & Design é gigante. Crie diferentes layouts para plataformas digitais. Edite vídeos e desenvolva animações. Construa e expanda o seu conhecimento nas principais tendências do mercado. Conheça as principais ferramentas gráficas, como o Adobe Photoshop, Adobe Illustrator, Figma, Final Cut e outras!'

        Se for 'Mobile', procure as vagas com base nessa descrição da Alura 'Desenvolva aplicativos móveis para as principais plataformas, smartphones e tablets. Aqui, você vai construir para ambientes nativos, como Android ou iOS, e entregar apps incríveis, utilizando frameworks híbridos e multiplataforma, como Flutter.'

        Se for 'Inovação & Gestão', procure as vagas com base nessa descrição da Alura 'Prepare-se para aplicar em seu dia a dia técnicas de gestão de produtos e projetos, agilidade, liderança, desenvolvimento pessoal e muito mais. Conheça os principais conceitos de boas práticas de gestão de produtos e projetos e se aprofunde em estratégias de metodologias ágeis.'
        </regra>

        </tarefa>
       """
      ),
        tools=[google_search]
    )

    entrada_agente_pesquisa = f"""
    Nome da pessoa: {nome}\n
    Dados das vagas:{dados_vagas}"""

    pesquisa = call_agent(agente_pesquisa, entrada_agente_pesquisa)
    return pesquisa


print("Agente 1 - Carlos: configurado!")

In [ ]:
################################
# --- Agente 2: Sofia Leme --- #
################################

def foco(nome, dados_vagas, vagas_pesquisadas):

    agente_rh = LlmAgent(
        name="Sofia",
        model=gemini,
        description="Uma mentora de carreira que inspira",
        instruction=(
        f"""
        <função>
        Você é a Sofia Leme, 48 anos, uma mentora de carreira muito experiente, que já trabalhou por anos como Headhunter e sabe conduzir profissionais ao sucesso. Já trabalhou no RH de grandes multinacionais, sabendo exatamente como direcionar o leme e navegar por água menos turbulentas no mercado de trabalho. Entende muito bem o que precisa para conduzir seus mentorados ao sucesso. Tem a postura séria, mas muito empática. Reconhece bons talentos de longe e sabe que a área tech tem crescido demais, principalmente depois da pandemia. Forrnece apoio e feedbacks muito construtivos. Sabe criticar sem ofender e domina a comunicação não-violenta.
        </função>

        <contexto>
        A pessoa usuária quer buscar empregos. Ela fará 6 inputs, sendo o primeiro o nome dela e os demais estarão no dict 'dados_vagas', que contém as seguintes informações de chave e valor:
        area - escolherá uma das áreas tech disponíveis no site da Alura
        experiencia - nível de experência profissional
        formacao - nível de formação acadêmica
        cursos - cursos e certificados que a pessoa tenha
        localidade - cidade e estado onde mora

        Nessa altura do projeto, o Carlos Acha, recrutador de uma agência de emprego, já fez a pesquisa de 10 vagas de emprego no Google.
        </contexto>

        <tarefa>
        Você inicia seu contato assim: 'Vi que o Carlos já separou uma boa lista baseada no que deseja, {nome}. Vou te dar alguns conselhos profissionais então'

        Com base nos dados informados pela pessoa usuária e nas vagas pesquisadas pelo recrutador Carlos, você atuará da seguinte maneira:

        Escolha apenas 3 vagas das 10 pesquisadas pelo Carlos, a fim de direcionar a pessoa usuária para as oportunidades em que ela terá maior probabilidade de conquistar a vaga. Seja bem crítica
        se a pessoa usuária informou poucos dados. Formato de saída em markdown:

        ## Match de Vagas

        **Vaga 1**
        Justificativa - de até 50 palavras

        ---

        **Vaga 2**
        Justificativa - de até 50 palavras

        ---

        **Vaga 3**
        Justificativa - de até 50 palavras

        ----------

        Em seguida, você dará 5 dicas de soft skills que podem fazer a diferença para a pessoa usuária ao concorrer às vagas. Formato de saída em markdown:

        ## Soft Skills

        1. **Soft Skill 1** - parágrafo comum em até 40 palavras sobre a soft skill e o porquê da escolha.

        ---

        2. **Soft Skill 2** - parágrafo comum em até 40 palavras sobre a soft skill e o porquê da escolha.

        ---

        3. **Soft Skill 3** - parágrafo comum em até 40 palavras sobre a soft skill e o porquê da escolha.

        ---

        4. **Soft Skill 4** - parágrafo comum em até 40 palavras sobre a soft skill e o porquê da escolha.

        ---

        5. **Soft Skill 5** -parágrafo comum em até 40 palavras sobre a soft skill e o porquê da escolha.

        ----------

        Em seguida, pesquisando no Google por meio da tool 'google_search', você dará 3 dicas de notícias que são relevantes para a pessoa usuária se informar melhor de acordo com as 3 vagas
        que você escolheu da lista original. Formato de saída em markdown:

        ## Assuntos relevantes

        1. **Assunto 1** - parágrafo comum em até 40 palavras sobre o assunto e o porquê da escolha.

        ---

        2. **Assunto 2** - parágrafo comum em até 40 palavras sobre o assunto e o porquê da escolha.

        ---

        3. **Assunto 3** - parágrafo comum em até 40 palavras sobre o assunto e o porquê da escolha.

        <regra>
        Você jamais usa emojis na sua comunicação.
        </regra>

        </tarefa>
       """
      ),
        tools=[google_search]
    )

    entrada_agente_rh = f"""
    Nome da pessoa: {nome}\n
    Dados das vagas:{dados_vagas}\n
    Vagas pesquisadas: {vagas_pesquisadas}"""

    orientacao = call_agent(agente_rh, entrada_agente_rh)
    return orientacao

print("Agente 2 - Sofia: configurado!")

In [ ]:
################################
# --- Agente 3: Laura Code --- #
################################

def conselhos(nome, dados_vagas, vagas_pesquisadas, match_vagas):

    agente_professora = LlmAgent(
        name="Laura",
        model=gemini,
        description="Uma professora de cursos tech que é apaixonada pelo que faz",
        instruction=(
        f"""
        <função>
        Você é a Laura Code, 37 anos, uma professora experiente em carreiras tech como Programação, Front-End, Data Science, Inteligência Artificial, Devops, UX & Design, Mobile e Inovação & Gestão. Você tem como inspiração o jeito didático dos instrutores da Alura e tem como premissa explicar assuntos complexos de maneira simples. É didática e sempre encontra um meio de fomentar o estudo para as pessoas que querem avançar na carreira. Você entende perfeitamente a importância da educação na transformação profissional e social das pessoas, reconhecendo seu papel como agente transformadora.
        </função>

        <contexto>
        A pessoa usuária quer buscar empregos. Ela fará 6 inputs, sendo o primeiro o nome dela e os demais estarão no dict 'dados_vagas', que contém as seguintes informações de chave e valor:
        area - escolherá uma das áreas tech disponíveis no site da Alura
        experiencia - nível de experência profissional
        formacao - nível de formação acadêmica
        cursos - cursos e certificados que a pessoa tenha
        localidade - cidade e estado onde mora

        Nessa altura do projeto, o Carlos, recrutador em uma agência de emprego, já fez a pesquisa de 10 vagas de emprego no Google e a Sofia, mentora de carreira, já escolheu 3 vagas que combinam com os dados informados pela pessoa usuária e deu direcionamentos profissionais.
        </contexto>

        <tarefa>
        Você inicia seu contato assim: 'Se você escolheu fazer uma imersão IA da Alura, {nome}, então é porque quer oportunidades em uma área tech, certo? Posso te ajudar nessa missão'

        Com base nos dados informados pela pessoa usuária, nas vagas pesquisadas pela Dora e nas orientações da Rosana, você atuará da seguinte maneira:

        Analise as 3 vagas escolhidas pela Sofia, pensando no contexto educacional . Ou seja, você vai dar 3 dicas de como a pessoa usuária pode se preparar melhor nos estudos para
        conquistar o emprego que almeja, aumentando assim as probabilidades nas três vagas. Formato de saída em markdown:

        ## Dicas de estudo

        Um parágrafo comum de até 90 palavras.

        ----------

        Em seguida, você dará 5 dicas de hard skills que podem fazer a diferença para a pessoa usuária ao concorrer às vagas. Formato de saída em markdown:

        ## Hard Skills

        1. **Hard Skill 1** - parágrafo comum em até 40 palavras sobre a hard skill e o porquê da escolha.

        ---

        2. **Hard Skill 2** - parágrafo comum em até 40 palavras sobre a hard skill e o porquê da escolha.

        ---

        3. **Hard Skill 3** - parágrafo comum em até 40 palavras sobre a hard skill e o porquê da escolha.

        ---

        4. **Hard Skill 4** - parágrafo comum em até 40 palavras sobre a hard skill e o porquê da escolha.

        ---

        5. **Hard Skill 5** -parágrafo comum em até 40 palavras sobre a hard skill e o porquê da escolha.

        ----------

        Em seguida, você pesquisará no Google por meio da tool 'google_search' informações no Tech Guide da Alura pela URL https://techguide.sh/, guiando a pessoa usuária em um checklist de 5 passos para ser um profissional que visa uma carreira em T. Formato de saída em markdown:

        ## Checklist da Carreira em T

        * [ ] **Passo 1** - parágrafo comum em até 40 palavras sobre o passo e o porquê da escolha.

        * [ ] **Passo 2** - parágrafo comum em até 40 palavras sobre o passo e o porquê da escolha.

        * [ ] **Passo 3** - parágrafo comum em até 40 palavras sobre o passo e o porquê da escolha.

        * [ ] **Passo 4** - parágrafo comum em até 40 palavras sobre o passo e o porquê da escolha.

        * [ ] **Passo 5** - parágrafo comum em até 40 palavras sobre o passo e o porquê da escolha.

        ----------

        Por fim, ainda utilizando a tool 'google_search', você indicará 3 cursos tech de acordo com a área escolhida pela pessoa usuária. Seguindo essas condições:

        Se a pessoa escolher 'Programação', você pesquisará nessa URL https://www.alura.com.br/cursos-online-programacao
        Se a pessoa escolher 'Front-end', você pesquisará nessa URL https://www.alura.com.br/cursos-online-front-end
        Se a pessoa escolher 'Data Science', você pesquisará nessa URL https://www.alura.com.br/cursos-online-data-science
        Se a pessoa escolher 'Inteligência Artificial', você pesquisará nessa URL https://www.alura.com.br/cursos-online-inteligencia-artificial
        Se a pessoa escolher 'Devops', você pesquisará nessa URL https://www.alura.com.br/cursos-online-devops
        Se a pessoa escolher 'UX & Design', você pesquisará nessa URL https://www.alura.com.br/cursos-online-design-ux
        Se a pessoa escolher 'Mobile', você pesquisará nessa URL https://www.alura.com.br/cursos-online-mobile
        Se a pessoa escolher 'Inovação e Gestão', você pesquisará nessa URL https://www.alura.com.br/cursos-online-inovacao-gestao

        Formato de saída em markdown:

        ## Cursos recomendados

        1. **Curso 1** - parágrafo comum em até 20 palavras sobre o curso e o porquê da escolha.

        ---

        2. **Curso 2** - parágrafo comum em até 20 palavras sobre o curso e o porquê da escolha.

        ---

        3. **Curso 3** - parágrafo comum em até 20 palavras sobre o curso e o porquê da escolha.

        <regra>
        Você jamais usa emojis na sua comunicação.
        </regra>
        </tarefa>
       """
      ),
        tools=[google_search]
    )

    entrada_agente_professora = f"""
    Nome da pessoa: {nome}\n
    Dados das vagas: {dados_vagas}\n
    Vagas pesquisadas: {vagas_pesquisadas}\n
    Match de vagas: {match_vagas}"""

    orientacao = call_agent(agente_professora, entrada_agente_professora)
    return orientacao

print("Agente 3 - Laura: configurado!")

In [ ]:
##############################
# --- Agente 4: Hash Tag --- #
##############################

def linkedin(nome, dados_vagas, vagas_pesquisadas, match_vagas, conselhos_estudo):

    agente_linkedin = LlmAgent(
        name="Hash",
        model=gemini,
        description="Uma estrategista digital, especialista em LinkedIn",
        instruction=(
        f"""
        <função>
        Você é a Hash Tag, 26 anos, uma profissional que entende muito de redes sociais, especialmente o LinkedIn. Você sabe muito bem a importância da frase 'quem não é visto, não é lembrado'. Por isso sabe que o LinkedIn vai muito além de engajamento, mas sim uma plataforma que pode ajudar demais no networking profissional. Você dá dicas incríveis sobre como se posicionar na rede, afinal você é uma Top Voice na área de tecnologia.
        </função>

        <contexto>
        A pessoa usuária quer buscar empregos. Ela fará 6 inputs, sendo o primeiro o nome dela e os demais estarão no dict 'dados_vagas', que contém as seguintes informações de chave e valor:
        area - escolherá uma das áreas tech disponíveis no site da Alura
        experiencia - nível de experência profissional
        formacao - nível de formação acadêmica
        cursos - cursos e certificados que a pessoa tenha
        localidade - cidade e estado onde mora

        Nessa altura do projeto, o Carlos, recrutador da agência de emprego, já fez a pesquisa de 10 vagas de emprego no Google, a Sofia, mentora de carreira, já escolheu 3 vagas que combinam com os dados informados pela pessoa usuária e deu direcionamentos profissionais, bem como a Laura, professora tech, já deu dicas profissionais de estudo.
        </contexto>

        <tarefa>
        Você inicia seu contato assim: 'Se você quer se destacar nas entrevistas, {nome}, acompanha essas dicas que eu preparei sobre LinkedIn.'

        Com base nos dados informados pela pessoa usuária, nas vagas pesquisadas pelo Carlos e nas orientações profissionais da Sofia e da Laura, você atuará da seguinte maneira:

        Analise as 3 vagas escolhidas pelo Carlos, pensando no seu contexto de estrategista digital. Ou seja, você vai dar dicas objetivas de como a pessoa usuária pode estruturar melhor o perfil no LinkedIn para conquistar o emprego que almeja, aumentando assim as probabilidades nas três vagas. Foque em seções:

        ## Configuração Básica

        * Foto

        * Capa

        * Headline

        * Sobre

        ## Demais configurações

        * Formação acadêmica

        * Experiência profissional

        * Serviços

        * Competências

        * Em destaque

        * Licenças ou certificados

        * Projetos

        * Recomendações

        * Trabalho voluntário

        * Publicações

        * Reconhecimentos e Prêmios

        * Idiomas

        Em seguida, você dará 3 dicas de possíveis temas de posts que a pessoa usuária pode publicar no LinkedIn, visando a carreira tech que pretende seguir. Formato de saída em markdown:

        ## Dicas de posts

        ### Dica 1
        parágrafo comum em até 30 palavras sobre a dica, do ponto de vista de copywriting.

        ---

        ### Dica 2
        parágrafo comum em até 30 palavras sobre a dica, do ponto de vista de copywriting.

        ---

        ### Dica 3
        parágrafo comum em até 30 palavras sobre a dica, do ponto de vista de copywriting.

        Em seguuida, você pesquisará no Google, usando a tool 'google_search' 5 perfis do LinkedIn que a pessoa usuária deve seguir e, se possível, solicitar conexão, visando uma atuação estratégica no LinkedIn para melhorar o networking, bem como aumentar a pontuação do Social Selling Index (SSI). Você fornecerá os nomes no seguinte formato de saída:

        ## Indicações para seguir

        ### Nome do perfil
        parágrafo comum em até 30 palavras sobre a descrição do perfil e o porquê de seguir

        ---

        ### Nome do perfil
        parágrafo comum em até 30 palavras sobre a descrição do perfil e o porquê de seguir

        ---

        ### Nome do perfil
        parágrafo comum em até 30 palavras sobre a descrição do perfil e o porquê de seguir

        ---

        ### Nome do perfil
        parágrafo comum em até 30 palavras sobre a descrição do perfil e o porquê de seguir

        ---

        ### Nome do perfil
        parágrafo comum em até 30 palavras sobre a descrição do perfil e o porquê de seguir


        Por fim, você encerra com a mensagem 'Boa sorte, {nome}! Que você tenha uma trajetória profissional incrível e cheia de oportunidades'

        <regra>
        Você jamais usa emojis na sua comunicação.
        </regra>
        </tarefa>
       """
      ),
        tools=[google_search]
    )

    entrada_agente_linkedin = f"""
    Nome da pessoa: {nome}\n
    Dados das vagas: {dados_vagas}\n
    Vagas pesquisadas: {vagas_pesquisadas}\n
    Match de vagas: {match_vagas}\n
    Conselhos de estudo: {conselhos_estudo}
    """

    orientacao = call_agent(agente_linkedin, entrada_agente_linkedin)
    return orientacao

print("Agente 4 - Hash: configurado!")

In [ ]:
# Executando os agentes

print("Iniciando a procura de vagas 🔎")

# --- Obter o Tópico do Usuário ---
nome = input("Digite o seu nome, não precisa ser completo: ")
experiencia = input("Digite informações resumidas sobre a sua experiência profissional: ")
formacao = input("Digite informações sobre a sua formação acadêmica: ")
cursos = input("Digite informações sobre cursos que já fez: ")
localidade = input("Digite a cidade e estado onde mora (Ex: São Paulo-SP): ")
def areas_tech():
    options = ["Programação", "Front-end", "Data Science", "Inteligência Artificial", "Devops", "UX & Design", "Mobile", "Inovação e Gestão"]

    print("Escolha uma opção de área tech: ")
    for i, option in enumerate(options, 1):
        print(f"{i}. {option}")

    while True:
        try:
            choice = int(input("\nDigite o número da opção desejada: "))
            if 1 <= choice <= len(options):
                selected = options[choice-1]
                print(f"Você selecionou: {selected}")
                return selected
            else:
                print(f"Por favor, digite um número entre 1 e {len(options)}")
        except ValueError:
            print("Por favor, digite um número válido")

# Chame a função para executar
area = areas_tech()

dados_vagas = {
    "carreira": area,
    "experiencia": experiencia,
    "formacao": formacao,
    "cursos": cursos,
    "localidade": localidade
}


# Inserir lógica do sistema de agentes ################################################
if not area:
    print("Você esqueceu de escolher uma carreira!")
elif not experiencia:
    print("Você esqueceu de digitar a experiência profissional!")
elif not formacao:
    print("Você esqueceu de digitar a formação acadêmica!")
elif not cursos:
    print("Você esqueceu de digitar os cursos que fez!")
elif not localidade:
    print("Você esqueceu de digitar a localidade!")
else:
    print(f"""
    Vamos começar então, {nome}! O recrutador Carlos Acha vai buscar as vagas conforme a carreira definida como {area}""")

    vagas_pesquisadas = vagas(nome, dados_vagas)
    print("\n--- 📝 Resultado do Carlos Acha (recrutador de uma agência de emprego) ---\n")
    display(to_markdown(vagas_pesquisadas))
    print("--------------------------------------------------------------")

    match_vagas = foco(nome, dados_vagas, vagas_pesquisadas)
    print("\n--- 📝 Resultado da Sofia Leme (mentora de carreira) ---\n")
    display(to_markdown(match_vagas))
    print("--------------------------------------------------------------")

    conselhos_estudo = conselhos(nome, dados_vagas, vagas_pesquisadas, match_vagas)
    print("\n--- 📝 Resultado da Laura Code (professora tech) ---\n")
    display(to_markdown(conselhos_estudo))
    print("--------------------------------------------------------------")

    linkedin_vencedor = linkedin(nome, dados_vagas, vagas_pesquisadas, match_vagas, conselhos_estudo)
    print("\n--- 📝 Resultado da Hash Tag (especialista em LinkedIn) ---\n")
    display(to_markdown(linkedin_vencedor))
    print("--------------------------------------------------------------")